In [1]:
 
import os
GLPK_FOLDER_PATH = "C:/Software/octave-9.2.0/mingw64/bin/"
os.environ["PATH"] += os.pathsep + str(GLPK_FOLDER_PATH)

In [1]:

import os


In [2]:
os.environ["PATH"] += os.pathsep + "C:/glpk-4.65/w64"

## GOING FOR LOOP 

In [ ]:
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, SolverFactory, minimize, value
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from BasicDefinitions import *

# Load data
df = pd.read_csv("D:/thesis/LinearExample/YearHourly.csv", header=None, index_col=0)
df.columns = ["Tout", "Qhouse", "Price el."]
df.index.name = "Time"
df.info()

tout = df["Tout"].to_numpy()
tflow = fTflow(tout)
qhouse = df["Qhouse"].to_numpy()
p = df["Price el."].to_numpy()
n = p.size

# Convert matrices to numerical values for better performance
e = np.eye(n)
s = np.tril(np.ones_like(e), 0)
c_scalar = float(4.2 * 500 / 3600)  # Convert C into scalar value
tinit = 30
m = s / c_scalar
r = tinit - np.dot(s, qhouse) / c_scalar  # Use np.dot for efficiency

# Define Pyomo Model
model = ConcreteModel()
model.qprod = Var(range(n), bounds=(0, Pth))
model.tbuffer = Var(range(n), bounds=(lambda m, i: tflow[i], Tmax))

# Cost coefficients
a = 3
cel = p / CoP(tout, tflow)
cte = p / CoP(tout, tflow + 1) - p / CoP(tout, tflow)

# Objective function
def objective_rule(model):
    return sum(cel[i] * model.qprod[i] + a * cte[i] * model.tbuffer[i] for i in range(n))

model.obj = Objective(rule=objective_rule, sense=minimize)

# Constraints for buffer temperature dynamics
def buffer_constraint(model, i):
    return sum(-s[i, j] / c_scalar * model.qprod[j] for j in range(n)) + model.tbuffer[i] == r[i]

model.buffer_constraints = Constraint(range(n), rule=buffer_constraint)

# Ensure total produced heat meets the demand
def demand_constraint(model):
    return sum(model.qprod[i] for i in range(n)) >= qhouse.sum()

model.demand_constraint = Constraint(rule=demand_constraint)

# Solve the problem
solver = SolverFactory('ipopt')
start = time.time()
result = solver.solve(model, tee=True)
end = time.time()

# Extract solution
qprod_opt = np.array([value(model.qprod[i]) for i in range(n)])
tbuffer_opt = np.array([value(model.tbuffer[i]) for i in range(n)])

# Output results
print(f"Runtime linear solver: {end - start:.2f} s")
print(f"Optimal total cost (with temperature costs): {value(model.obj):.2f} €")
costs_excluding_temperature = np.sum(cel * qprod_opt)
print(f"Costs excluding temperature costs: {costs_excluding_temperature:.2f} €")

# Visualization
plt.figure(figsize=(12, 6))
plt.plot(range(n), qprod_opt, marker='o', linestyle='-', color='b', label='Optimized Qprod')
plt.axhline(y=np.mean(qprod_opt), color='r', linestyle='--', label='Mean Qprod')
plt.xlabel("Time Step")
plt.ylabel("Optimized Qprod")
plt.title("Optimization Results Over Time - Pyomo")
plt.legend()
plt.grid()
plt.show()

# Histogram of optimized values
plt.figure(figsize=(10, 5))
plt.hist(qprod_opt, bins=20, color='skyblue', edgecolor='black')
plt.xlabel("Optimized Qprod")
plt.ylabel("Frequency")
plt.title("Distribution of Optimized Qprod")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# Scatter plot to observe any patterns
plt.figure(figsize=(10, 5))
plt.scatter(range(n), qprod_opt, color='purple', label='Optimized Qprod')
plt.xlabel("Time Step")
plt.ylabel("Optimized Qprod")
plt.title("Scatter Plot of Optimization Results")
plt.legend()
plt.grid()
plt.show()


<class 'pandas.core.frame.DataFrame'>
Index: 8760 entries, 2021-01-01 00:00:00 to 2021-12-31 23:00:00
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Tout       8760 non-null   float64
 1   Qhouse     8760 non-null   float64
 2   Price el.  8760 non-null   float64
dtypes: float64(3)
memory usage: 273.8+ KB


In [5]:
pip install pyomo


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: C:\Users\himan\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [9]:
import os

In [10]:
os.environ["PATH"] += os.pathsep + "C:/Ipopt/bin"

In [ ]:
















from pyomo.environ import ConcreteModel, Var, Objective, Constraint, SolverFactory, minimize, value
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from BasicDefinitions import *

# Load data
df = pd.read_csv("D:/thesis/LinearExample/YearHourly.csv", header=None, index_col=0)
df.columns = ["Tout", "Qhouse", "Price el."]
df.index.name = "Time"
df.info()

tout = df["Tout"].to_numpy()
tflow = fTflow(tout)
qhouse = df["Qhouse"].to_numpy()
p = df["Price el."].to_numpy()
n = p.size

# Convert matrices to numerical values for better performance
e = np.eye(n)
s = np.tril(np.ones_like(e), 0)
c_scalar = float(4.2 * 500 / 3600)  # Convert C into scalar value
tinit = 30
m = s / c_scalar
r = tinit - np.dot(s, qhouse) / c_scalar  # Use np.dot for efficiency

# Define Pyomo Model
model = ConcreteModel()
model.qprod = Var(range(n), bounds=(0, Pth))
model.tbuffer = Var(range(n), bounds=(lambda m, i: tflow[i], Tmax))

# Cost coefficients
a = 3
cel = p / CoP(tout, tflow)
cte = p / CoP(tout, tflow + 1) - p / CoP(tout, tflow)

# Objective function
def objective_rule(model):
    return np.dot(cel, [model.qprod[i] for i in range(n)]) + a * np.dot(cte, [model.tbuffer[i] for i in range(n)])

model.obj = Objective(rule=objective_rule, sense=minimize)

# Constraints for buffer temperature dynamics
def buffer_constraint(model, i):
    return np.dot(-s[i, :], [model.qprod[j] for j in range(n)]) / c_scalar + model.tbuffer[i] == r[i]

model.buffer_constraints = Constraint(range(n), rule=buffer_constraint)

# Ensure total produced heat meets the demand
def demand_constraint(model):
    return np.dot([model.qprod[i] for i in range(n)], np.ones(n)) >= qhouse.sum()

model.demand_constraint = Constraint(rule=demand_constraint)

# Solve the problem
solver = SolverFactory('ipopt')
start = time.time()
result = solver.solve(model, tee=True)
end = time.time()

# Extract solution
qprod_opt = np.array([value(model.qprod[i]) for i in range(n)])
tbuffer_opt = np.array([value(model.tbuffer[i]) for i in range(n)])

# Output results
print(f"Runtime linear solver: {end - start:.2f} s")
print(f"Optimal total cost (with temperature costs): {value(model.obj):.2f} €")
costs_excluding_temperature = np.dot(cel, qprod_opt)
print(f"Costs excluding temperature costs: {costs_excluding_temperature:.2f} €")

# Visualization
plt.figure(figsize=(12, 6))
plt.plot(range(n), qprod_opt, marker='o', linestyle='-', color='b', label='Optimized Qprod')
plt.axhline(y=np.mean(qprod_opt), color='r', linestyle='--', label='Mean Qprod')
plt.xlabel("Time Step")
plt.ylabel("Optimized Qprod")
plt.title("Optimization Results Over Time - Pyomo")
plt.legend()
plt.grid()
plt.show()

# Histogram of optimized values
plt.figure(figsize=(10, 5))
plt.hist(qprod_opt, bins=20, color='skyblue', edgecolor='black')
plt.xlabel("Optimized Qprod")
plt.ylabel("Frequency")
plt.title("Distribution of Optimized Qprod")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# Scatter plot to observe any patterns
plt.figure(figsize=(10, 5))
plt.scatter(range(n), qprod_opt, color='purple', label='Optimized Qprod')
plt.xlabel("Time Step")
plt.ylabel("Optimized Qprod")
plt.title("Scatter Plot of Optimization Results")
plt.legend()
plt.grid()
plt.show()


<class 'pandas.core.frame.DataFrame'>
Index: 8760 entries, 2021-01-01 00:00:00 to 2021-12-31 23:00:00
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Tout       8760 non-null   float64
 1   Qhouse     8760 non-null   float64
 2   Price el.  8760 non-null   float64
dtypes: float64(3)
memory usage: 273.8+ KB
